In [1]:
import pandas as pd
import datetime

In [2]:
#month -> season
def season(month) : 
    if month>=3 and month <= 5 : return 0 #Spring
    elif month>=6 and month <= 8 : return 1 #Summber
    elif month>=9 and month <= 11 : return 2 #Fall
    else : return 3 #Winter

# 1. 데이터 읽어오기

In [3]:
tashu = pd.read_csv('data/tashu.csv' )
weather = pd.read_csv('data/weather.csv' )
station = pd.read_csv('data/station.csv' )

# 2. Data preprocessing

## 2.1 NaN Data & station > 144  drop

In [4]:
tashu.dropna(axis= 0)
tashu = tashu[tashu['RENT_STATION'] <= 144 ]
tashu = tashu[tashu['RETURN_STATION'] <= 144 ]

## 2.2 Date형식 변경

In [5]:
temp_weather = weather
#data 분리
temp_rent_tashu = tashu[['RENT_STATION' , 'RENT_DATE']]
temp_return_tashu = tashu[['RETURN_STATION' , 'RETURN_DATE']]

In [6]:
#to_Datetime
temp_rent_tashu['RENT_DATE']= pd.to_datetime(temp_rent_tashu['RENT_DATE'])

c:\users\s\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
temp_return_tashu['RETURN_DATE']= pd.to_datetime(temp_return_tashu['RETURN_DATE'])

c:\users\s\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
temp_weather['TIME']= pd.to_datetime(temp_weather['TIME'])

## 2.2.1 Date -> year, month, day, hour

### 2.2.1.1 RentDate

In [9]:

#rent
rent_year = []
rent_month = []
rent_day = []
rent_hour = []

for i in temp_rent_tashu['RENT_DATE'] :
    rent_year.append(i.year)
    rent_month.append(i.month)
    rent_day.append(i.day)
    rent_hour.append(i.hour)
    
temp_rent_tashu['YEAR'] = rent_year
temp_rent_tashu['MONTH'] = rent_month
temp_rent_tashu['DAY'] = rent_day
temp_rent_tashu['HOUR'] = rent_hour


c:\users\s\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### 2.2.1.2 ReturnDate

In [10]:

return_year = []
return_month = []
return_day = []
return_hour = []

for j in temp_return_tashu['RETURN_DATE'] : 
    return_year.append(j.year)
    return_month.append(j.month)
    return_day.append(j.day)
    return_hour.append(j.hour)
    
temp_return_tashu['YEAR'] = return_year
temp_return_tashu['MONTH'] = return_month
temp_return_tashu['DAY'] = return_day
temp_return_tashu['HOUR'] = return_hour

c:\users\s\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [11]:
del temp_rent_tashu['RENT_DATE']
del temp_return_tashu['RETURN_DATE']


#### RETURN_DATE 2016 DELETE

In [12]:
temp_return_tashu = temp_return_tashu[temp_return_tashu['YEAR'] < 2016]

### 2.2.1.3 WeatherTime

In [13]:
#Weather Time -> YEAR , MONTH, DAY , HOUR , WEEKDAY , SEASION
weather_year = []
weather_month = []
weather_day = []
weather_hour = []
weather_weekday = []
weather_season = []


for i in temp_weather['TIME'] :
    weather_year.append(i.year)
    weather_month.append(i.month)
    weather_day.append(i.day)
    weather_hour.append(i.hour)
    weather_weekday.append(i.weekday())
    weather_season.append(season(i.month))

temp_weather['YEAR'] = weather_year
temp_weather['MONTH'] = weather_month
temp_weather['DAY'] = weather_day
temp_weather['HOUR'] = weather_hour
temp_weather['WEEKDAY'] = weather_weekday
temp_weather['SEASON'] = weather_season

In [14]:
del temp_weather['TIME']

## 2.3 Merge ( station_num , weather )

In [15]:
from pandas import DataFrame
station_num = DataFrame(station['번호'])


In [16]:
total_rent_df = pd.merge(station_num.assign(dummy = 1), temp_weather.assign(dummy = 1), on = 'dummy').drop('dummy', axis=1)
total_return_df = pd.merge(station_num.assign(dummy = 1), temp_weather.assign(dummy = 1), on = 'dummy').drop('dummy', axis=1)


In [17]:
total_rent_df = total_rent_df.rename(columns={"번호":"RENT_STATION"})
total_return_df = total_return_df.rename(columns={"번호":"RETURN_STATION"})

## 2.4 Groupby -> count

### 2.4.1 Rent_count

In [18]:
rent_tashu_rentcount = temp_rent_tashu.groupby(['RENT_STATION', 'YEAR', 'MONTH' ,'DAY' ,'HOUR']).size().to_frame('RENTCOUNT').reset_index()

In [19]:
rent_tashu_rentcount

,RENT_STATION,YEAR,MONTH,DAY,HOUR,RENTCOUNT
0,1.0,2013,1,1,13,3
1,1.0,2013,1,2,7,2
2,1.0,2013,1,2,11,1
3,1.0,2013,1,2,18,1
4,1.0,2013,1,3,11,1
5,1.0,2013,1,4,18,1
6,1.0,2013,1,5,7,1
7,1.0,2013,1,5,14,1
8,1.0,2013,1,5,15,2
9,1.0,2013,1,6,14,1


### 2.4.2 Return_count

In [20]:
return_tashu_returncount = temp_return_tashu.groupby(['RETURN_STATION', 'YEAR', 'MONTH' ,'DAY' ,'HOUR']).size().to_frame('RETURNCOUNT').reset_index()

In [21]:
return_tashu_returncount

,RETURN_STATION,YEAR,MONTH,DAY,HOUR,RETURNCOUNT
0,1.0,2013,1,1,13,1
1,1.0,2013,1,2,7,1
2,1.0,2013,1,2,17,1
3,1.0,2013,1,2,23,2
4,1.0,2013,1,3,8,1
5,1.0,2013,1,3,13,1
6,1.0,2013,1,4,23,1
7,1.0,2013,1,5,8,1
8,1.0,2013,1,5,14,1
9,1.0,2013,1,5,15,1


## 2.5 Merge( total , count)

### 2.5.1 total_rent, rent_count

In [22]:
total_rent_df = pd.merge(total_rent_df , rent_tashu_rentcount , how='outer')
total_rent_df['RENTCOUNT'] = total_rent_df['RENTCOUNT'].fillna(0)

### 2.5.2 total_return , return_count

In [24]:
total_return_df = pd.merge(total_return_df , return_tashu_returncount , how='outer')
total_return_df['RETURNCOUNT'] = total_return_df['RETURNCOUNT'].fillna(0)

In [25]:
total_rent_df

,RENT_STATION,TEMPERATURE,RAINFALL,WINDSPEED,HUMLDITY,SNOWFALL,YEAR,MONTH,DAY,HOUR,WEEKDAY,SEASON,RENTCOUNT
0,1,-8.8,NaN,0.1,90.0,8.8,2013,1,1,0,1,3,0.0
1,1,-8.5,NaN,0.9,90.0,8.8,2013,1,1,1,1,3,0.0
2,1,-8.5,NaN,1.0,89.0,8.8,2013,1,1,2,1,3,0.0
3,1,-9.0,NaN,0.7,91.0,8.8,2013,1,1,3,1,3,0.0
4,1,-9.1,NaN,0.6,92.0,8.8,2013,1,1,4,1,3,0.0
5,1,-9.4,NaN,0.5,92.0,8.8,2013,1,1,5,1,3,0.0
6,1,-9.0,NaN,1.4,93.0,8.8,2013,1,1,6,1,3,0.0
7,1,-8.3,NaN,0.2,94.0,9.6,2013,1,1,7,1,3,0.0
8,1,-7.3,NaN,1.7,96.0,10.5,2013,1,1,8,1,3,0.0
9,1,-6.4,2.2,0.9,96.0,12.8,2013,1,1,9,1,3,0.0


In [26]:
total_return_df

,RETURN_STATION,TEMPERATURE,RAINFALL,WINDSPEED,HUMLDITY,SNOWFALL,YEAR,MONTH,DAY,HOUR,WEEKDAY,SEASON,RETURNCOUNT
0,1,-8.8,NaN,0.1,90.0,8.8,2013,1,1,0,1,3,0.0
1,1,-8.5,NaN,0.9,90.0,8.8,2013,1,1,1,1,3,0.0
2,1,-8.5,NaN,1.0,89.0,8.8,2013,1,1,2,1,3,0.0
3,1,-9.0,NaN,0.7,91.0,8.8,2013,1,1,3,1,3,0.0
4,1,-9.1,NaN,0.6,92.0,8.8,2013,1,1,4,1,3,0.0
5,1,-9.4,NaN,0.5,92.0,8.8,2013,1,1,5,1,3,0.0
6,1,-9.0,NaN,1.4,93.0,8.8,2013,1,1,6,1,3,0.0
7,1,-8.3,NaN,0.2,94.0,9.6,2013,1,1,7,1,3,0.0
8,1,-7.3,NaN,1.7,96.0,10.5,2013,1,1,8,1,3,0.0
9,1,-6.4,2.2,0.9,96.0,12.8,2013,1,1,9,1,3,0.0


In [29]:
total_rent_df.to_csv('data2/total_rent.csv')
total_return_df.to_csv('data2/total_return.csv')
